In [ ]:
# Classify CK+ moderate images into ModEm folder
import os

for root, dirs, files in os.walk("./cohn-kanade-images2"):
    filelist=[]
    for name in files:
        filename = os.path.join(root, name)
        if '.png' in filename:
            filelist.append(filename)
    if len(filelist) > 0:
        modfile = filelist[int(len(filelist)/2)]
        !cp $modfile ModEm
        print

In [ ]:
# Classify CK+ Neutral and Expression images into respective folders
import os
for root, dirs, files in os.walk("./cohn-kanade-images2"):
    filelist=[]
    for name in files:
        filename = os.path.join(root, name)
        if '.png' in filename:
            filelist.append(filename)
    if len(filelist) > 0:
        neutralfile = filelist[0]
        !cp $neutralfile Neutral
        expressfile = filelist[-1]
        !cp $expressfile StrEm
        print

In [40]:
# WORKING VERSION OF COMBINED DRAFTS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate normalized Euclidean Differences array
# Generate target values array
# Feed to SVM, evaulate with Cross Validation
import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    for d in dets:
        shape = predictor(image, d)
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)

"""MAIN PROGRAM BEGINS"""
# Get paths to face images and emotion labels
imagedir = []
labeldir = []
root, dirs, files = os.walk("./cohn-kanade-images2",).__next__()
for d in dirs:
    imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))

# initialize variables
dataset = []
NLA = []
ELA = []

# look over all directories of individuals S001 through S999
for d in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',d)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',d,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            faceimage = os.path.join(curdir,files[-1])

            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',d,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()
                    
                    # append label to dataset
                    dataset.append([baseimage, faceimage, targetlabel])

"""68-POINT LANDMARK DETECTION MODEL"""
# Get array of Euclidean differences between neutral and expression 68-landmark arrays
tempLDA = []
for imgIndex in range(len(dataset)):
    nImgName = dataset[imgIndex][0]
    eImgName = dataset[imgIndex][1]
    nLA = get68LMA(nImgName)
    eLA = get68LMA(eImgName)
    NLA.append(nLA)
    ELA.append(eLA)
    tempLDA.append(getEucDist(nLA, eLA))
LDA = np.asmatrix(tempLDA)

# Create target value array for expression images
tempTargetValues = []
for imgSet in range(len(dataset)):
    tempTargetValues.append(dataset[imgSet][2])
targetValues = np.asarray(tempTargetValues)

# # Feed LDA and targetValues into SVM, use cross validation, generate confusion matrix
# x_train, x_test, y_train, y_test = cross_validation.train_test_split(LDA, targetValues, test_size=0.4, random_state=2)
# normCLF = svm.SVC(kernel='linear').fit(x_train, y_train)
# predictions = normCLF.predict(x_test)
# cmatrix = confusion_matrix(y_test, predictions)

# # Format CMatrix to show percentages, rounded to 1 decimal point
# np.set_printoptions(precision=1)
# rowSums = np.sum(cmatrix, axis=1)
# pCMatrix = np.divide(cmatrix*100., rowSums, dtype=float)
# print pCMatrix


"""PHYSIOLOGICALLY BASED LANDMARK DETECTION MODEL"""
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', PDA) #Save PDA as human-readable format file

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='poly',probability=True).fit(x_train2, y_train2)
predictions2 = physCLF.predict(x_test2)
cMatrix2 = confusion_matrix(y_test2, predictions2)

# Format CMatrix2 to show percentages, already rounded to 1 decimal point
# rowSums2 = np.sum(cMatrix2, axis=1)
# pCMatrix2 = np.divide(cMatrix2*100., rowSums2, dtype=float)
print(cMatrix2)

joblib.dump(polyCLF, 'poly7.pkl') 

[[11  2  1  0  0  0  1]
 [ 0  7  0  0  0  0  0]
 [ 1  0 19  0  0  0  0]
 [ 1  1  0  7  1  1  0]
 [ 0  0  1  0 27  0  0]
 [ 1  5  0  2  1  6  0]
 [ 0  2  0  2  0  0 31]]


['poly7.pkl',
 'poly7.pkl_01.npy',
 'poly7.pkl_02.npy',
 'poly7.pkl_03.npy',
 'poly7.pkl_04.npy',
 'poly7.pkl_05.npy',
 'poly7.pkl_06.npy',
 'poly7.pkl_07.npy',
 'poly7.pkl_08.npy',
 'poly7.pkl_09.npy',
 'poly7.pkl_10.npy',
 'poly7.pkl_11.npy']

In [8]:
# SET UP METHODS FOR SVM

import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    shape = predictor(image, dets[0])
    for index in range(0, 68):
        point = shape.part(index)
        tuplepoint = (point.x, point.y)
        LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPhysDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)



In [36]:
# Copied from last year
#
#

# WORKING VERSION OF COMBINED DRAFTS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate normalized Euclidean Differences array
# Generate target values array
# Feed to SVM, evaulate with Cross Validation
import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    for d in dets:
        shape = predictor(image, d)
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)

"""MAIN PROGRAM BEGINS"""
# Get paths to face images and emotion labels
imagedir = []
labeldir = []
root, dirs, files = os.walk("./cohn-kanade-images2",).__next__()
for d in dirs:
    imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))

# initialize variables
dataset = []
NLA = []
ELA = []
MLA = []

# look over all directories of individuals S001 through S999
for d in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',d)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',d,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            midimage = os.path.join(curdir,files[int(len(files)/2)])
            faceimage = os.path.join(curdir,files[-1])

            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',d,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()
                    
                    # append label to dataset
                    dataset.append([baseimage, faceimage, midimage, targetlabel])

"""68-POINT LANDMARK DETECTION MODEL"""
# Get array of Euclidean differences between neutral and expression 68-landmark arrays
tempLDA = []
for imgIndex in range(len(dataset)):
    nImgName = dataset[imgIndex][0]
    eImgName = dataset[imgIndex][1]
    mImgName = dataset[imgIndex][2]
    nLA = get68LMA(nImgName)
    eLA = get68LMA(eImgName)
    mLA = get68LMA(mImgName)
    NLA.append(nLA)
    ELA.append(eLA)
    MLA.append(mLA)
#    tempLDA.append(getEucDist(nLA, eLA))
#LDA = np.asmatrix(tempLDA)

# Create target value array for expression images
tempTargetValues = []
for imgSet in range(len(dataset)):
    if dataset[imgSet][3] == 0:
        tempTargetValues.append(0.)
        tempTargetValues.append(0.)    
    else:
        tempTargetValues.append(dataset[imgSet][3])
        tempTargetValues.append(dataset[imgSet][3])    
targetValues = np.asarray(tempTargetValues)

# # Feed LDA and targetValues into SVM, use cross validation, generate confusion matrix
# x_train, x_test, y_train, y_test = cross_validation.train_test_split(LDA, targetValues, test_size=0.4, random_state=2)
# normCLF = svm.SVC(kernel='linear').fit(x_train, y_train)
# predictions = normCLF.predict(x_test)
# cmatrix = confusion_matrix(y_test, predictions)

# # Format CMatrix to show percentages, rounded to 1 decimal point
# np.set_printoptions(precision=1)
# rowSums = np.sum(cmatrix, axis=1)
# pCMatrix = np.divide(cmatrix*100., rowSums, dtype=float)
# print pCMatrix


"""PHYSIOLOGICALLY BASED LANDMARK DETECTION MODEL"""
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
    tempPDA.append(getPDA(NLA[i])-getPDA(MLA[i]))
PDA = np.asmatrix(tempPDA)
#np.savetxt('PDAfile', PDA) #Save PDA as human-readable format file

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='rbf').fit(x_train2, y_train2)
predictions2 = physCLF.predict(x_test2)
cMatrix2 = confusion_matrix(y_test2, predictions2)

# Format CMatrix2 to show percentages, already rounded to 1 decimal point
# rowSums2 = np.sum(cMatrix2, axis=1)
# pCMatrix2 = np.divide(cMatrix2*100., rowSums2, dtype=float)
print(cMatrix2)

[[ 7  1  2  0  0  1 31]
 [ 0  4  0  0  1  2  7]
 [ 0  0 12  0  0  0 30]
 [ 0  0  0  0  0  0 18]
 [ 0  0  1  0 35  0 31]
 [ 0  0  1  0  0  3 13]
 [ 0  0  1  0  0  0 61]]


In [35]:
dataset[1]

['./cohn-kanade-images2/S010/002/S010_002_00000001.png',
 './cohn-kanade-images2/S010/002/S010_002_00000014.png',
 './cohn-kanade-images2/S010/002/S010_002_00000008.png',
 7.0]

In [28]:
getPDA(NLA[0])

array([  29.15475947,  168.19036833,  141.03191128,   30.59411708,
         64.89992296,   89.28605714,  141.15594213,  120.2081528 ,
         31.06444913,   68.26419266,  151.60804728,   40.16217126,
         29.73213749])

In [31]:
getPDA(ELA[0])

array([  28.7923601 ,  168.        ,  141.35416513,   26.83281573,
         61.84658438,   88.32326987,  132.69890731,  118.37651794,
         33.13608305,   71.55417528,  145.49914089,   37.64306045,
         26.62705391])

In [32]:
getPDA(MLA[0])

array([  29.15475947,  168.        ,  141.42842713,   27.31300057,
         62.64982043,   88.88756943,  134.89254983,  118.53269591,
         32.01562119,   70.23531875,  147.24808997,   37.64306045,
         29.15475947])

In [3]:
# RBF
# SVM CLASSIFIERS FOR STRONG EMOTION EXPRESSIONS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate array of normalized Euclidean physiological distances
# Generate target values array
# Feed to SVM, evaulate with Cross Validation

import os

# Initialize variables
#imagedir = []
labeldir = []
dataset = []

# Get paths to face images and emotion labels
root, dirs, files = os.walk("./cohn-kanade-images2").__next__()
for d in dirs:
    #imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))
           
# Look over all directories of individuals S001 through S999
for person in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',person)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',person,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            midimage= os.path.join(curdir,files[int(len(files)/2)])
            faceimage = os.path.join(curdir,files[-1])
            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',person,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()                    
                    # append label to dataset
                    if int(targetlabel)!=2:
                        dataset.append([baseimage, midimage, faceimage, targetlabel])

"""GET DATA TO TRAIN MODEL"""
#1: Get array of corresponding differences between 13 distances of base images and strong/moderate expression images
tempPDA = []
for imgIndex in range(len(dataset)):
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][2])))
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][1])))
    tempPDA.append([0]*13)
physDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', physDA) #Save PDA as human-readable format file

#2: Create target value array for expression images
tempTargVals = []
for imgSet in range(len(dataset)):
    if dataset[imgSet][3]==0:                      #if expression sequence is neutral
        tempTargVals.append(0.)
        tempTargVals.append(0.)
    else:                                         #if expression sequence has emotion
        tempTargVals.append(dataset[imgSet][3]+20.)
        tempTargVals.append(dataset[imgSet][3]+10.)
    tempTargVals.append(0.)
targetValues = np.asarray(tempTargVals)

"""FEED DATA TO TRAIN MODEL"""
# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(physDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='rbf').fit(x_train, y_train)
predictions = physCLF.predict(x_test)
cMatrix = confusion_matrix(y_test, predictions)

"""EVAULUATE USING READABLE CONFUSION MATRIX"""
# Format CMatrix to show percentages, already rounded to 1 decimal point
rowSums = np.sum(cMatrix, axis=1)
pCMatrix = np.divide(cMatrix*100., rowSums, dtype=float)
print(pCMatrix)

[[ 100.            0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.            0.
     0.        ]
 [   0.            9.09090909    5.            0.            3.7037037
     0.           48.27586207   44.44444444    0.            0.            0.
     0.            0.        ]
 [   0.            0.           25.            0.            3.7037037
     0.           44.82758621    0.            4.76190476    0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.            0.
    44.82758621    0.            0.            0.            0.            0.
     0.        ]
 [   0.            0.            5.            0.           44.44444444
     0.           37.93103448    0.            0.            0.
    10.71428571    0.            0.        ]
 [   0.            0.            5.            0.            0.            0.
    31.03448276    0.            0.

In [4]:
from sklearn.externals import joblib
joblib.dump(physCLF, 'physModelRBF.pkl') 


['physModelRBF.pkl',
 'physModelRBF.pkl_01.npy',
 'physModelRBF.pkl_02.npy',
 'physModelRBF.pkl_03.npy',
 'physModelRBF.pkl_04.npy',
 'physModelRBF.pkl_05.npy',
 'physModelRBF.pkl_06.npy',
 'physModelRBF.pkl_07.npy',
 'physModelRBF.pkl_08.npy',
 'physModelRBF.pkl_09.npy',
 'physModelRBF.pkl_10.npy',
 'physModelRBF.pkl_11.npy']

In [ ]:
# Capture video from robot camera
# Show video in color
# Display box around face
# Does NOT display facial landmarks for each face
# SLOW Face Tracking

import cv2
import dlib
import nxt.locator
from nxt.motor import *

cap = cv2.VideoCapture(1) # capture video from robot camera
cap.set(3,640)
cap.set(4,480)
b = nxt.locator.find_one_brick() # connect to NXT brick

# # Initialize variables for dLib
# predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor(predictor_path)

images=[]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Detect faces and initialize variables for next step
    dets = detector(frame, 1)
    
    # Draw a rectangle around the faces
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
        cv2.rectangle(frame, (0,0), (1,1), (0, 255, 0), 2)
        facecenter=rect.center()
        if facecenter.y>280:
            Motor(b, PORT_B).turn(10, 10)
        elif facecenter.y<200:
            Motor(b, PORT_B).turn(-10, 10)
        if facecenter.x>360:
            Motor(b, PORT_A).turn(10, 10)
        elif facecenter.x<280:
            Motor(b, PORT_A).turn(-10, 10)
        break

    # Display frame, press 'q' to exit video window
#     cv2.imshow('frame',frame)
#      if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# When everything done, release the capture
cap.release()
# cv2.destroyAllWindows()

In [4]:
# Method to get 68 landmarks for IMAGE passed to the function
def getI68LMA(img):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    
    # Detect face(s)
    dets = detector(img, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    if len(dets)>0:
        shape = predictor(img, dets[0])
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
        return LMA

In [7]:
# Capture video
# Classify emotion
# Output the emotion

import cv2
import dlib
import numpy as np
from sklearn.externals import joblib

cap = cv2.VideoCapture(0) # capture video from robot camera
cap.set(3,640)
cap.set(4,480)
loaded = joblib.load('physModel.pkl')
detector = dlib.get_frontal_face_detector()

images=[]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    dets = detector(frame, 1)
    
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    cv2.imshow('frame',frame)

    # Get physiological feature vector for current frame
    images.append(getPhysDA(getI68LMA(frame)))
    
    if len(images)>2:
        images.pop(0)
    # Classify expression in image
    if len(images)>1:
        prob=loaded.predict(np.asarray(images[0]-images[1])) # Get array of probabilities
        index = np.argmax(prob)# Get position of highest probability
        print(prob)
        
        # Print the emotion corresponding to the position
        if index==0:
            print("Neutral")
        elif index==1:
            print("Moderate Anger")
        elif index==2:
            print("Moderate Disgust")
        elif index==3:
            print("Moderate Fear")
        elif index==4:
            print("Moderate Happiness")
        elif index==5:
            print("Moderate Sadness")
        elif index==6:
            print("Moderate Surprise")
        elif index==7:
            print("Strong Anger")
        elif index==8:
            print("Strong Disgust")
        elif index==9:
            print("Strong Fear")
        elif index==10:
            print("Strong Happiness")
        elif index==11:
            print("Strong Sadness")
        elif index==12:
            print("Strong Surprise")
            
    # Break out of while loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
        
# Release video capture
cap.release()
cv2.destroyAllWindows()

[ 17.]
Neutral
[ 16.]
Neutral
[ 11.]
Neutral
[ 0.]
Neutral
[ 16.]
Neutral
[ 0.]
Neutral
[ 0.]
Neutral
[ 0.]
Neutral
[ 11.]
Neutral
[ 16.]
Neutral
[ 0.]
Neutral
[ 0.]
Neutral
[ 0.]
Neutral
[ 16.]
Neutral
[ 0.]
Neutral
[ 16.]
Neutral
[ 17.]
Neutral
[ 14.]
Neutral
[ 13.]
Neutral


KeyboardInterrupt: 

In [2]:
cap.release()

In [10]:
cv2.destroyAllWindows()

In [ ]:
# SET UP METHODS FOR SVM

import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix    
from sklearn.externals import joblib

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    shape = predictor(image, dets[0])
    for index in range(0, 68):
        point = shape.part(index)
        tuplepoint = (point.x, point.y)
        LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPhysDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)
# Method to get 68 landmarks for IMAGE passed to the function
def getI68LMA(img):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    
    # Detect face(s)
    dets = detector(img, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    if len(dets)>0:
        shape = predictor(img, dets[0])
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
        return LMA
    
    

cap = cv2.VideoCapture(0) # capture video from robot camera
cap.set(3,640)
cap.set(4,480)
model = joblib.load('physModel.pkl')
detector = dlib.get_frontal_face_detector()

images=[]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    dets = detector(frame, 1)
    
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    cv2.imshow('frame',frame)

    # Get physiological feature vector for current frame
    if len(dets)>0:
        images.append(getPhysDA(getI68LMA(frame)))
    
        if len(images)>2:
            images.pop(0)
        # Classify expression in image
        if len(images)>1:
            index=model.predict(np.array(images[0]-images[1]).reshape(1,-1)) # Get array of probabilities

            # Print the emotion corresponding to the position
            if index==0:
                print("Neutral")
            elif index==11:
                print("Moderate Anger")
            elif index==13:
                print("Moderate Disgust")
            elif index==14:
                print("Moderate Fear")
            elif index==15:
                print("Moderate Happiness")
            elif index==16:
                print("Moderate Sadness")
            elif index==17:
                print("Moderate Surprise")
            elif index==21:
                print("Strong Anger")
            elif index==23:
                print("Strong Disgust")
            elif index==24:
                print("Strong Fear")
            elif index==25:
                print("Strong Happiness")
            elif index==26:
                print("Strong Sadness")
            elif index==27:
                print("Strong Surprise")
            
    # Break out of while loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
        
# Release video capture
cap.release()
cv2.destroyAllWindows()

In [2]:
cv2.destroyAllWindows()

In [4]:
# RBF 3/4 MOD IMG
# SVM CLASSIFIERS FOR STRONG EMOTION EXPRESSIONS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate array of normalized Euclidean physiological distances
# Generate target values array
# Feed to SVM, evaulate with Cross Validation

import os
from sklearn.externals import joblib

# Initialize variables
#imagedir = []
labeldir = []
dataset = []

# Get paths to face images and emotion labels
root, dirs, files = os.walk("./cohn-kanade-images2").__next__()
for d in dirs:
    #imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))
           
# Look over all directories of individuals S001 through S999
for person in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',person)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',person,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            midimage= os.path.join(curdir,files[int(len(files)*3/4)])
            faceimage = os.path.join(curdir,files[-1])
            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',person,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()                    
                    # append label to dataset
                    if int(targetlabel)!=2:
                        dataset.append([baseimage, midimage, faceimage, targetlabel])

"""GET DATA TO TRAIN MODEL"""
#1: Get array of corresponding differences between 13 distances of base images and strong/moderate expression images
tempPDA = []
for imgIndex in range(len(dataset)):
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][2])))
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][1])))
    tempPDA.append([0]*13)
physDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', physDA) #Save PDA as human-readable format file

#2: Create target value array for expression images
tempTargVals = []
for imgSet in range(len(dataset)):
    if dataset[imgSet][3]==0:                      #if expression sequence is neutral
        tempTargVals.append(0.)
        tempTargVals.append(0.)
    else:                                         #if expression sequence has emotion
        tempTargVals.append(dataset[imgSet][3]+20.)
        tempTargVals.append(dataset[imgSet][3]+10.)
    tempTargVals.append(0.)
targetValues = np.asarray(tempTargVals)

"""FEED DATA TO TRAIN MODEL"""
# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(physDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='rbf').fit(x_train, y_train)
predictions = physCLF.predict(x_test)
cMatrix = confusion_matrix(y_test, predictions)

"""EVAULUATE USING READABLE CONFUSION MATRIX"""
# Format CMatrix to show percentages, already rounded to 1 decimal point
# rowSums = np.sum(cMatrix, axis=1)
# pCMatrix = np.divide(cMatrix*100., rowSums, dtype=float)
print(cMatrix)
joblib.dump(physCLF, 'RBF34.pkl') 


[[138   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  13   9   0   0   0   0   0]
 [  0   0   2   0   0   0  11   0   7   0   0   0   0]
 [  0   0   0   0   0   0  13   0   0   0   0   0   0]
 [  0   0   0   0   7   0   5   0   0   0  15   0   0]
 [  0   0   0   0   0   0  10   0   0   0   0   1   0]
 [  0   0   0   0   0   0  24   0   0   0   0   0   5]
 [  0   0   0   0   0   0   9   0   0   0   0   0   0]
 [  0   0  12   0   0   0   8   1   0   0   0   0   0]
 [  0   0   0   0   0   0  12   0   0   0   0   0   0]
 [  0   0   0   0  17   0   8   0   0   0   3   0   0]
 [  0   0   0   0   0   0   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0  35   0   0   0   0   0   0]]


['RBF34.pkl',
 'RBF34.pkl_01.npy',
 'RBF34.pkl_02.npy',
 'RBF34.pkl_03.npy',
 'RBF34.pkl_04.npy',
 'RBF34.pkl_05.npy',
 'RBF34.pkl_06.npy',
 'RBF34.pkl_07.npy',
 'RBF34.pkl_08.npy',
 'RBF34.pkl_09.npy',
 'RBF34.pkl_10.npy',
 'RBF34.pkl_11.npy']

In [5]:
# RBF 3/4 MOD IMG
# SVM CLASSIFIERS FOR STRONG EMOTION EXPRESSIONS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate array of normalized Euclidean physiological distances
# Generate target values array
# Feed to SVM, evaulate with Cross Validation

import os
from sklearn.externals import joblib

# Initialize variables
#imagedir = []
labeldir = []
dataset = []

# Get paths to face images and emotion labels
root, dirs, files = os.walk("./cohn-kanade-images2").__next__()
for d in dirs:
    #imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))
           
# Look over all directories of individuals S001 through S999
for person in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',person)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',person,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            midimage= os.path.join(curdir,files[int(len(files)*3/4)])
            faceimage = os.path.join(curdir,files[-1])
            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',person,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()                    
                    # append label to dataset
                    if int(targetlabel)!=2:
                        dataset.append([baseimage, midimage, faceimage, targetlabel])

"""GET DATA TO TRAIN MODEL"""
#1: Get array of corresponding differences between 13 distances of base images and strong/moderate expression images
tempPDA = []
for imgIndex in range(len(dataset)):
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][2])))
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][1])))
    tempPDA.append([0.]*13)
physDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', physDA) #Save PDA as human-readable format file

#2: Create target value array for expression images
tempTargVals = []
for imgSet in range(len(dataset)):
    if dataset[imgSet][3]==0:                      #if expression sequence is neutral
        tempTargVals.append(0.)
        tempTargVals.append(0.)
    else:                                         #if expression sequence has emotion
        tempTargVals.append(dataset[imgSet][3]+20.)
        tempTargVals.append(dataset[imgSet][3]+10.)
    tempTargVals.append(0.)
targetValues = np.asarray(tempTargVals)

"""FEED DATA TO TRAIN MODEL"""
# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(physDA, targetValues, test_size=0.4, random_state=3)
lin34CLF = svm.SVC(kernel='linear').fit(x_train, y_train)
predictions = physCLF.predict(x_test)
L34cMatrix = confusion_matrix(y_test, predictions)

"""EVAULUATE USING READABLE CONFUSION MATRIX"""
# Format CMatrix to show percentages, already rounded to 1 decimal point
# rowSums = np.sum(cMatrix, axis=1)
# pCMatrix = np.divide(cMatrix*100., rowSums, dtype=float)
print(L34cMatrix)
joblib.dump(lin34CLF, 'linear34.pkl') 


[[138   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  13   9   0   0   0   0   0]
 [  0   0   2   0   0   0  11   0   7   0   0   0   0]
 [  0   0   0   0   0   0  13   0   0   0   0   0   0]
 [  0   0   0   0   7   0   5   0   0   0  15   0   0]
 [  0   0   0   0   0   0  10   0   0   0   0   1   0]
 [  0   0   0   0   0   0  24   0   0   0   0   0   5]
 [  0   0   0   0   0   0   9   0   0   0   0   0   0]
 [  0   0  12   0   0   0   8   1   0   0   0   0   0]
 [  0   0   0   0   0   0  12   0   0   0   0   0   0]
 [  0   0   0   0  17   0   8   0   0   0   3   0   0]
 [  0   0   0   0   0   0   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0  35   0   0   0   0   0   0]]


['linear34.pkl',
 'linear34.pkl_01.npy',
 'linear34.pkl_02.npy',
 'linear34.pkl_03.npy',
 'linear34.pkl_04.npy',
 'linear34.pkl_05.npy',
 'linear34.pkl_06.npy',
 'linear34.pkl_07.npy',
 'linear34.pkl_08.npy',
 'linear34.pkl_09.npy',
 'linear34.pkl_10.npy',
 'linear34.pkl_11.npy']

In [39]:
# RBF 3/4 MOD IMG
# SVM CLASSIFIERS FOR STRONG EMOTION EXPRESSIONS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate array of normalized Euclidean physiological distances
# Generate target values array
# Feed to SVM, evaulate with Cross Validation

import os
from sklearn.externals import joblib

# Initialize variables
#imagedir = []
labeldir = []
dataset = []

# Get paths to face images and emotion labels
root, dirs, files = os.walk("./cohn-kanade-images2").__next__()
for d in dirs:
    #imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))
           
# Look over all directories of individuals S001 through S999
for person in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',person)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',person,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            midimage= os.path.join(curdir,files[int(len(files)/2.)])
            faceimage = os.path.join(curdir,files[-1])
            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',person,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()                    
                    # append label to dataset
                    if int(targetlabel)!=2:
                        dataset.append([baseimage, midimage, faceimage, targetlabel])

"""GET DATA TO TRAIN MODEL"""
#1: Get array of corresponding differences between 13 distances of base images and strong/moderate expression images
tempPDA = []
for imgIndex in range(len(dataset)):
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][2])))
    tempPDA.append(getPhysDA(get68LMA(dataset[imgIndex][0]))-getPhysDA(get68LMA(dataset[imgIndex][1])))
physDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', physDA) #Save PDA as human-readable format file

#2: Create target value array for expression images
tempTargVals = []
for imgSet in range(len(dataset)):
    tempTargVals.append(dataset[imgSet][3]+10.)
    tempTargVals.append(dataset[imgSet][3])
targetValues = np.asarray(tempTargVals)

"""FEED DATA TO TRAIN MODEL"""
# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(physDA, targetValues, test_size=0.4, random_state=3)
polyCLF = svm.SVC(kernel='poly',probability=True).fit(x_train, y_train)
predictions = polyCLF.predict(x_test)
polycMatrix = confusion_matrix(y_test, predictions)

"""EVAULUATE USING READABLE CONFUSION MATRIX"""
# Format CMatrix to show percentages, already rounded to 1 decimal point
# rowSums = np.sum(cMatrix, axis=1)
# pCMatrix = np.divide(cMatrix*100., rowSums, dtype=float)
print(polycMatrix)
joblib.dump(polyCLF, 'poly13.pkl') 


[[ 9  2  0  1  7  3  6  1  0  0  0  0]
 [ 0 17  0  0  2  0  1  5  0  0  0  0]
 [ 1  0  3  1  1  2  0  0  2  1  1  0]
 [ 0  1  3 19  0  1  0  0  1 10  0  0]
 [ 0  0  0  0  1  1  0  0  1  0  3  0]
 [ 0  0  3  0  0 10  0  0  2  0  1  5]
 [11  2  0  0  0  0  3  3  0  0  0  0]
 [ 1  6  0  1  1  0  1  7  0  1  0  0]
 [ 0  0  4  0  0  1  0  0  2  0  1  0]
 [ 0  0  0 16  0  0  0  0  1 20  0  0]
 [ 0  0  1  0  4  0  0  0  1  0  2  0]
 [ 0  0  2  1  0  8  0  0  2  0  0 17]]


['poly13.pkl',
 'poly13.pkl_01.npy',
 'poly13.pkl_02.npy',
 'poly13.pkl_03.npy',
 'poly13.pkl_04.npy',
 'poly13.pkl_05.npy',
 'poly13.pkl_06.npy',
 'poly13.pkl_07.npy',
 'poly13.pkl_08.npy',
 'poly13.pkl_09.npy',
 'poly13.pkl_10.npy',
 'poly13.pkl_11.npy']